In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
# import multiprocessing
# os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
# import run_script
# import parser_arguments
# import importlib
# importlib.reload(run_script)
import datetime
from pathlib import Path
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2p03_BARE2"]
# chip_names = ["ET2p01_BARE10"]
chip_fignames = chip_names
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM5"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60]
ws_addresses = [None, None, None, None]

fig_outdir = Path('../ETROC-figures')
# fig_outdir = Path('/media/daq/X9/DESY_Jun2024_AIDA/ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1"),("1","1"),("1","1"), ("1","1")])

In [ ]:
# Calibrate PLL
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)
# Calibrate FC for all I2C
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

# Config chips

### Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [ ]:
# board_RFSels = {
#     0x63: 0x0,
# }
# if(board_RFSels is not None):
#     time.sleep(5)
#     for chip_address in chip_addresses:
#         if chip_address not in board_RFSels:
#             continue
#         chip = i2c_conn.get_chip_i2c_connection(chip_address)
#         row_indexer_handle,_,_ = chip.get_indexer("row")
#         column_indexer_handle,_,_ = chip.get_indexer("column")
#         broadcast_handle,_,_ = chip.get_indexer("broadcast")
#         column_indexer_handle.set(0)
#         row_indexer_handle.set(0)
#         chip.read_all_block("ETROC2", "Pixel Config")
#         RFSel_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "RFSel")
#         RFSel_handle.set(hex(board_RFSels[chip_address])) # default is '0b10'
#         broadcast_handle.set(True)
#         chip.write_all_block("ETROC2", "Pixel Config")
#         print(f"Changing RFSel to {hex(board_RFSels[chip_address])} for chip: {hex(chip_address)}")

In [ ]:
# (WS Prep Pixel and Peri) - (Offsets) - (disable & auto_cal all pixels) - (disable default all pixels) - (auto_TH_CAL) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
i2c_conn.config_chips('00100101')
# i2c_conn.config_chips('00001000') # disable and more fun things
# i2c_conn.config_chips('00000100') # set basic peripherals
# i2c_conn.config_chips('00010000') # calibrate only
# i2c_conn.config_chips('00000001') # pixel ID check (I2C check)
# i2c_conn.config_chips('10000000') # ws init
# i2c_conn.disable_all_pixels(chip_address=chip_addresses[1])

## Set power mode to high if currents are too low

In [ ]:
# full_col_list, full_row_list = np.meshgrid(np.arange(16),np.arange(16))
# full_scan_list = list(zip(full_row_list.flatten(),full_col_list.flatten()))
# for address in chip_addresses:
#     i2c_conn.set_power_mode_scan_list(address, full_scan_list, 'high')

## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

In [ ]:
# i2c_conn.BL_df[96].pivot(index='row', columns='col', values='baseline')

In [ ]:
# histdir = Path('../ETROC-History/')
# histdir = Path('/media/daq/X9/DESY_Jun2024_AIDA/ETROC-History/')
histdir = Path('/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_CC_Spirale3')
histdir.mkdir(exist_ok=True)
histfile = histdir / 'BaselineHistory.sqlite'
figure_title = "-24.5C, 0 MRad, A: 1.2V, D: 0.95V"
i2c_conn.save_baselines(chip_fignames, fig_title=figure_title, fig_path=fig_path, histdir=histdir, histfile=histfile, save_notes=figure_title)

In [ ]:
config_outdir = Path('../ETROC-Data')
config_outdir = config_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
config_outdir = config_outdir / "ChipConfig"
config_outdir.mkdir(parents=True, exist_ok=True)

i2c_conn.dump_config(config_outdir, "PreTID_ETROC2p3_BARE9_RoomT_Aug02")

In [ ]:
# for chip_address, chip_name in zip(chip_addresses, chip_names):
#     i2c_conn.save_auto_cal_BL_map(chip_address, chip_name, "")
#     i2c_conn.save_auto_cal_NW_map(chip_address, chip_name, "")

In [ ]:
# for chip_address, chip_name in zip(chip_addresses, chip_names):
#     i2c_conn.load_auto_cal_BL_map(chip_address, chip_name, "")
#     i2c_conn.load_auto_cal_NW_map(chip_address, chip_name, "")

# Define pixels of interest

In [ ]:
# row_list = [15, 15, 15, 15]
# col_list = [6, 7, 8, 9]
# row_list = [14, 14, 14, 14]
# col_list = [6, 7, 8, 9]
# row_list = [9, 3, 12]
# col_list = [3, 8, 11]
# row_list = [15, 15]
# col_list = [6, 9]
# row_list = [14, 14, 15, 15]
# col_list = [6, 9, 6, 9]
# row_list = [14, 14, 14, 14, 15, 15, 15, 15]
# col_list = [6, 7, 8, 9, 6, 7, 8, 9]
# row_list = np.arange(16)
# col_list = np.full_like(row_list,6)
# row_list = np.full_like(col_list,13)
col_list = [7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8]
row_list = [15,14,13,12,3,2,1,0, 15,14,13,12,3,2,1,0]

scan_list = list(zip(row_list, col_list))

# col_list, row_list = np.meshgrid(np.arange(16),np.arange(16))
# scan_list = list(zip(row_list.flatten(),col_list.flatten()))

print(scan_list)

### Enable pixels of Interest

In [ ]:
i2c_conn.enable_select_pixels_in_chips(scan_list,QInjEn=True,Bypass_THCal=True,specified_addresses=chip_addresses[:],power_mode='high',verbose=False)

### Set offset for all boards

In [ ]:
offset = 0x0a
for chip_address,chip_name in zip(chip_addresses,chip_names):
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    i2c_conn.set_chip_offsets(chip_address, offset=offset, chip=chip, pixel_list=scan_list, verbose=False)
    del chip

#### Set QSel or TH_offset for all boards

In [ ]:
offset = 0x0a
charge = 0x0a
for chip_address,chip_name in zip(chip_addresses,chip_names):
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in scan_list:
        # print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        i2c_conn.pixel_decoded_register_write("QSel", format(charge, '05b'), chip)
        i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)
        i2c_conn.pixel_decoded_register_write("QInjEn", '1', chip)
    del row_indexer_handle, column_indexer_handle, chip
time.sleep(2)

### QInj Pixels

In [ ]:
qinj_pixels = {
    0x60: [ # (row, col)
        (6, 6),
    ],
    0x61: [ # (row, col)
        (7, 7),
    ],
    0x62: [ # (row, col)
        (8, 8),
    ],
    0x63: [ # (row, col)
        (9, 9),
    ],
}

for chip_address in chip_addresses:
    if chip_address not in qinj_pixels:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in qinj_pixels[chip_address]:
        print(f"Enabling QInj of chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        # i2c_conn.pixel_decoded_register_write("QSel", format(0x19, '05b'), chip)
        i2c_conn.pixel_decoded_register_write("QInjEn", '1', chip)


### Set Different offsets for all boards

In [ ]:
board_offsets = {
    0x60: 0x0a,
    0x61: 0x0a,
    0x62: 0x0a,
    0x63: 0x0a,
}

for chip_address,chip_name in zip(chip_addresses,chip_names):
    if chip_address not in board_offsets:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    i2c_conn.set_chip_offsets(chip_address, offset=board_offsets[chip_address], chip=chip, pixel_list=scan_list, verbose=False)
    # row_indexer_handle,_,_ = chip.get_indexer("row")
    # column_indexer_handle,_,_ = chip.get_indexer("column")
    # for row, col in scan_list:
    #     print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
    #     column_indexer_handle.set(col)
    #     row_indexer_handle.set(row)
    #     i2c_conn.pixel_decoded_register_write("QSel", format(0x14, '05b'), chip)
    #     # i2c_conn.pixel_decoded_register_write("TH_offset", format(board_offsets[chip_address], '06b'), chip)
    # del row_indexer_handle, column_indexer_handle
    del chip

### Offset Pixel per board

In [ ]:
offset_pixels = {
    0x60: { # (row, col): offset
        (10, 8): 15,
    }
}

for chip_address in chip_addresses:
    if chip_address not in offset_pixels:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in offset_pixels[chip_address]:
        print(f"Setting offset of chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        i2c_conn.pixel_decoded_register_write("TH_offset", format(offset_pixels[chip_address][(row, col)], '06b'), chip)

### Remove noisy pixels from Trigger

In [ ]:
noisy_pixels = {
    0x60: [       # (row, col)
        (6, 7),
    ],
}

for chip_address in chip_addresses:
    if chip_address not in noisy_pixels:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in noisy_pixels[chip_address]:
        print(f"Masking from trigger of chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)

        # i2c_conn.enable_pixel_modular(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle, QInjEn=False, Bypass_THCal=False, triggerWindow=True, cbWindow=True)
        # i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)
        i2c_conn.pixel_decoded_register_write("DAC", format(1023, '010b'), chip)
        # i2c_conn.disable_pixel(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
        i2c_conn.pixel_decoded_register_write("disTrigPath", "1", chip)
        # if(chip_address==0x62): 
        #     i2c_conn.disable_pixel(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
            # i2c_conn.pixel_decoded_register_write("DAC", format(1023, '010b'), chip)
        #     i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", chip)



### Disable Pixels of Interest

In [ ]:
for chip_address in chip_addresses[:]:
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row,col in scan_list:
        i2c_conn.disable_pixel(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
    del chip, row_indexer_handle, column_indexer_handle

# Calibrate FC for all I2C

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

# Calibrate PLL

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)

# eFuse

In [ ]:
chip_address = chip_addresses[0]
chip = i2c_conn.get_chip_i2c_connection(chip_address)

chip.read_all_block("ETROC2", "Peripheral Config")
handle_EFuse_EnClk = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_EnClk")
handle_EFuse_TCKHP = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_TCKHP")
handle_EFuse_Mode = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_Mode")
handle_EFuse_Rstn = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_Rstn")
handle_EFuse_Start = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_Start")
handle_EFuse_Bypass = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_Bypass")
handle_EFuse_Prog = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "EFuse_Prog")

chip.read_all_block("ETROC2", "Peripheral Status")
handle_EFuseQ = chip.get_decoded_display_var("ETROC2", "Peripheral Status", "EFuseQ")

handle_EFuse_EnClk.set(hex(0b1))
handle_EFuse_Bypass.set(hex(0b0))
handle_EFuse_Rstn.set(hex(0b1))
handle_EFuse_Start.set(hex(0b0))

handle_EFuse_Prog.set(hex(0xE5AE0009))
# handle_EFuse_Prog.set(hex(0xFAF00008))
# handle_EFuse_Prog.set(hex(0x5C9A0007))
# handle_EFuse_Prog.set(hex(0x43C40006))
# handle_EFuse_Prog.set(hex(0x62260005))
# handle_EFuse_Prog.set(hex(0x7D780004))
# handle_EFuse_Prog.set(hex(0x21E20003)) # 0x10f10003
# handle_EFuse_Prog.set(hex(0x3EBC0002))
# handle_EFuse_Prog.set(hex(0x1F5E0001))

# 01011100100110100000000000000111
# handle_EFuse_Prog.set(hex(0b00100001111000100000000000000011))
# handle_EFuse_Prog.set(hex(0b00000000000000000000000000000001))
# handle_EFuse_Prog.set(hex(0b00010000111100010000000000000011))


handle_EFuse_Mode.set(hex(0b10))
handle_EFuse_TCKHP.set(hex(0x4))
chip.write_all_block("ETROC2", "Peripheral Config")
chip.read_all_block("ETROC2", "Peripheral Config")

In [ ]:
# EFuse Read

# handle_EFuse_Rstn.set('1')
# chip.write_all_block("ETROC2", "Peripheral Config")
handle_EFuse_Rstn.set(hex(0b0))
chip.write_all_block("ETROC2", "Peripheral Config")
handle_EFuse_Rstn.set(hex(0b1))
chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Rstn.set('0')
# chip.write_all_block("ETROC2", "Peripheral Config")

# handle_EFuse_Start.set('0')
# chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Start.set('1')
# chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Start.set('0')
# chip.write_all_block("ETROC2", "Peripheral Config")

handle_EFuse_Mode.set(hex(0b10))
chip.write_all_block("ETROC2", "Peripheral Config")

chip.read_all_block("ETROC2", "Peripheral Config")
print(
" EF CLK (Act High)- ",                handle_EFuse_EnClk.get(),
"\n EF TCKHP- ",                       handle_EFuse_TCKHP.get(),
"\n EF Mode (10 - read, 01-write)- ",  handle_EFuse_Mode.get(),
"\n EF Rstn (Act Low)- ",              handle_EFuse_Rstn.get(),
"\n EF Start (Act High to Program)- ", handle_EFuse_Start.get(),
"\n EF ByPass (Act High)- ",           handle_EFuse_Bypass.get(),
"\n EF Prog- ",                        handle_EFuse_Prog.get(),
"\n Chip ID (EF Prog [LSB 17 bits])- ",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):#06x}",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):>017b}\n\n",
f"EFProg Intd for chip {hex(chip_address)}: {handle_EFuse_Prog.get()}, {int(handle_EFuse_Prog.get(), 0):032b}"
) 

chip.read_all_block("ETROC2", "Peripheral Status")
print(f" EFuseQ Read for chip {hex(chip_address)}: {handle_EFuseQ.get()}, {int(handle_EFuseQ.get(), 0):032b}")

In [ ]:
# EFuse Write

# handle_EFuse_Mode.set(hex(0b01))
# chip.write_all_block("ETROC2", "Peripheral Config")

handle_EFuse_Rstn.set(hex(0b0))
chip.write_all_block("ETROC2", "Peripheral Config")
# time.sleep(2)
handle_EFuse_Rstn.set(hex(0b1))
chip.write_all_block("ETROC2", "Peripheral Config")

In [ ]:
# handle_EFuse_EnClk.set(hex(0b0))
# chip.write_all_block("ETROC2", "Peripheral Config")
# chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_EnClk")
# handle_EFuse_EnClk.set(hex(0b1))
# chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_EnClk")
# chip.write_all_block("ETROC2", "Peripheral Config")
for i in tqdm(range(1000)):
    handle_EFuse_Mode.set(hex(0b01))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Mode")
    handle_EFuse_TCKHP.set(hex((i%14)+1))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_TCKHP")
    handle_EFuse_Rstn.set(hex(0b0))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Rstn")
    # chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Rstn.set(hex(0b1))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Rstn")
    handle_EFuse_Start.set(hex(0b0))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Start")

    handle_EFuse_Start.set(hex(0b1))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Start")
    # time.sleep(2)
    # handle_EFuse_Start.set(hex(0b1))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Start.set(hex(0b0))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Start")
    # chip.write_all_block("ETROC2", "Peripheral Config")
    # time.sleep(0.05)
    handle_EFuse_Mode.set(hex(0b10))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Mode")

# 
# handle_EFuse_Start.set(hex(0b1))
# chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Start.set('1')
# chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Start.set('0')
# chip.write_all_block("ETROC2", "Peripheral Config")

# handle_EFuse_Mode.set(hex(0b10))
# chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Mode")
# chip.write_all_block("ETROC2", "Peripheral Config")

chip.read_all_block("ETROC2", "Peripheral Config")
print(
" EF CLK (Act High)- ",                handle_EFuse_EnClk.get(),
"\n EF TCKHP- ",                       handle_EFuse_TCKHP.get(),
"\n EF Mode (10 - read, 01-write)- ",  handle_EFuse_Mode.get(),
"\n EF Rstn (Act Low)- ",              handle_EFuse_Rstn.get(),
"\n EF Start (Act High to Program)- ", handle_EFuse_Start.get(),
"\n EF ByPass (Act High)- ",           handle_EFuse_Bypass.get(),
"\n EF Prog- ",                        handle_EFuse_Prog.get(),
"\n Chip ID (EF Prog [LSB 17 bits])- ",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):#06x}",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):>017b}\n\n",
f"EFProg Intd for chip {hex(chip_address)}: {handle_EFuse_Prog.get()}, {int(handle_EFuse_Prog.get(), 0):032b}"
) 

chip.read_all_block("ETROC2", "Peripheral Status")
print(f" EFuseQ Read for chip {hex(chip_address)}: {handle_EFuseQ.get()}, {int(handle_EFuseQ.get(), 0):032b}")

In [ ]:
# handle_EFuse_EnClk.set(hex(0b0))
# chip.write_all_block("ETROC2", "Peripheral Config")
# chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_EnClk")
# handle_EFuse_EnClk.set(hex(0b1))
# chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_EnClk")
# chip.write_all_block("ETROC2", "Peripheral Config")
for i in tqdm(range(100)):
    handle_EFuse_Mode.set(hex(0b01))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Mode")
    handle_EFuse_TCKHP.set(hex((i%14)+1))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_TCKHP")
    handle_EFuse_Rstn.set(hex(0b0))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Rstn")
    # chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Rstn.set(hex(0b1))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Rstn")
    handle_EFuse_Start.set(hex(0b0))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Start")

    handle_EFuse_Start.set(hex(0b1))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Start")
    # time.sleep(2)
    # handle_EFuse_Start.set(hex(0b1))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Start.set(hex(0b0))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Start")
    # chip.write_all_block("ETROC2", "Peripheral Config")
    # time.sleep(0.05)
    handle_EFuse_Mode.set(hex(0b10))
    chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Mode")

# 
# handle_EFuse_Start.set(hex(0b1))
# chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Start.set('1')
# chip.write_all_block("ETROC2", "Peripheral Config")
# handle_EFuse_Start.set('0')
# chip.write_all_block("ETROC2", "Peripheral Config")

# handle_EFuse_Mode.set(hex(0b10))
# chip.write_decoded_value("ETROC2","Peripheral Config","EFuse_Mode")
# chip.write_all_block("ETROC2", "Peripheral Config")

chip.read_all_block("ETROC2", "Peripheral Config")
print(
" EF CLK (Act High)- ",                handle_EFuse_EnClk.get(),
"\n EF TCKHP- ",                       handle_EFuse_TCKHP.get(),
"\n EF Mode (10 - read, 01-write)- ",  handle_EFuse_Mode.get(),
"\n EF Rstn (Act Low)- ",              handle_EFuse_Rstn.get(),
"\n EF Start (Act High to Program)- ", handle_EFuse_Start.get(),
"\n EF ByPass (Act High)- ",           handle_EFuse_Bypass.get(),
"\n EF Prog- ",                        handle_EFuse_Prog.get(),
"\n Chip ID (EF Prog [LSB 17 bits])- ",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):#06x}",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):>017b}\n\n",
f"EFProg Intd for chip {hex(chip_address)}: {handle_EFuse_Prog.get()}, {int(handle_EFuse_Prog.get(), 0):032b}"
) 

chip.read_all_block("ETROC2", "Peripheral Status")
print(f" EFuseQ Read for chip {hex(chip_address)}: {handle_EFuseQ.get()}, {int(handle_EFuseQ.get(), 0):032b}")

In [ ]:
# EFuse Write
counter = 0
while counter<10:
    counter+=1
    print("Attempt", counter)

    handle_EFuse_Mode.set(hex(0b01))
    chip.write_all_block("ETROC2", "Peripheral Config")
    chip.read_all_block("ETROC2", "Peripheral Config")
    print(
    " EF Mode (10 - read, 01-write)- ",  handle_EFuse_Mode.get(),
    ) 

    # handle_EFuse_Mode.set(hex(0b01))
    # chip.write_all_block("ETROC2", "Peripheral Config")
    # chip.read_all_block("ETROC2", "Peripheral Config")

    handle_EFuse_Rstn.set('1')
    chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Rstn.set('0')
    chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Rstn.set('1')
    chip.write_all_block("ETROC2", "Peripheral Config")


    handle_EFuse_Start.set('0')
    chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Start.set('1')
    chip.write_all_block("ETROC2", "Peripheral Config")
    handle_EFuse_Start.set('0')
    chip.write_all_block("ETROC2", "Peripheral Config")


    handle_EFuse_Mode.set(hex(0b10))
    chip.write_all_block("ETROC2", "Peripheral Config")
    chip.read_all_block("ETROC2", "Peripheral Config")

    print(
    " EF CLK (Act High)- ",                handle_EFuse_EnClk.get(),
    "\n EF TCKHP- ",                       handle_EFuse_TCKHP.get(),
    "\n EF Mode (10 - read, 01-write)- ",  handle_EFuse_Mode.get(),
    "\n EF Rstn (Act Low)- ",              handle_EFuse_Rstn.get(),
    "\n EF Start (Act High to Program)- ", handle_EFuse_Start.get(),
    "\n EF ByPass (Act High)- ",           handle_EFuse_Bypass.get(),
    "\n EF Prog- ",                        handle_EFuse_Prog.get(),
    "\n Chip ID (EF Prog [LSB 17 bits])- ",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):#06x}",f"{(int(handle_EFuse_Prog.get(), 0) & 0x0000ffff):>017b}\n\n",
    f"EFProg Intd for chip {hex(chip_address)}: {handle_EFuse_Prog.get()}, {int(handle_EFuse_Prog.get(), 0):032b}"
    ) 

    chip.read_all_block("ETROC2", "Peripheral Status")
    print(f" EFuseQ Read for chip {hex(chip_address)}: {handle_EFuseQ.get()}, {int(handle_EFuseQ.get(), 0):032b}")

    if (int(handle_EFuseQ.get(), 0)!=0): break

# Disconnect I2C Device

## DO THIS IF YOU PLAN TO DO NOISE STUDIES

In [ ]:
del i2c_conn